In [8]:
from linkedin import Linkedin
import json
import pysolr
from datetime import datetime
import pandas as pd
from string import ascii_uppercase
from random import randint
import sys
import string
from time import sleep

In [37]:
Maximum_profile = 5
Maximum_search = 20
count = 50
start = 13
## change the range number to the count of total linkedin account
search_count_per_acc = {k:0 for k,v in zip(ascii_uppercase,(v for v in range(3)))}
profile_count_per_acc = {k:0 for k,v in zip(ascii_uppercase,(v for v in range(3)))}
myAcc = ''
accounts = None
start_profileId = 0
region= {"geoUrn":'', "offset":''}
current_region= '102502717'
count_change_acc = 0

In [11]:
## to index new accounts add the login info into the account.json file
def index_acc():
    acc_array= []
    with open("account.json","r") as f:
        accs = json.loads(f.read())
        
        for acc,letter in zip(accs, ascii_uppercase):
            acc_array.append({"user": acc['user'],
                            "password": acc['password'],
                            "AccIdentifier": letter
                             })
        
    solr = pysolr.Solr("http://192.168.4.217:8983/solr/accounts", always_commit=True, timeout=10)   
    solr.add(acc_array)

In [4]:
def querry_acc():
    index_acc()
    global accounts
    solr = pysolr.Solr("http://192.168.4.217:8983/solr/accounts", always_commit=True, timeout=10)

    print('accounts info are retrieved')
    accounts = solr.search('*').docs

In [24]:
def login(index=0):
    global myAcc, count_change_acc
    api = Linkedin(accounts[index]['user'], accounts[index]['password'])
    myAcc = accounts[index]['AccIdentifier']
#     count_change_acc +=1
    print('user {} is logged in successfully'.format(myAcc))
    crawl_profile()

In [25]:
def change_acc():
    index = -1
    for i,acc in enumerate(accounts):
        if acc['AccIdentifier'] == myAcc:
            if i == len(accounts)-1 :
                'print all accounts have used their maximum limits'
                sys.exit()
            else:
                index= i
#     if count_change_acc == len(accounts):
#         'print all accounts have used their maximum limits'
#         sys.exit()
        
    login(index+1)

In [26]:
def index_profiles_id(profiles_id):
    data = list()
    for i in profiles_id:
        data.append(dict({"id": i['urn_id'],
            "distance": i['distance'],
            "public_id": i['public_id'],
            "tracking_id": i['tracking_id']
            }))
    
    solr = pysolr.Solr("http://192.168.4.217:8983/solr/profileId/", always_commit=True, timeout=10)
    solr.add(data)
    print('profile_ids have successfully indexed')


In [27]:
def index_profile(profile):
    solr = pysolr.Solr('http://192.168.4.217:8983/solr/person',always_commit=True, timeout=10) 
    solr.add(profile)
    print('profile info has successfully indexed')

In [68]:
def index_region(profile):
    try:
        solr = pysolr.Solr("http://192.168.4.217:8983/solr/region", always_commit=True,  timeout=100)
        solr.add([{"geoLocationName": profile['geoLocationName'],
                   "geoLocation":  profile['geoLocation']['geoUrn'].split(':')[3], 
                   "offset" : "0"
                 }])
        print('geoLocation has successfully indexed')
        
    except:
        print("geoLocation is not available for this user")

In [48]:
def data_processing(profile):
    if profile['education'] is not None:
        for i in range(len(profile['education'])):
            unwanted = set(profile['education'][i].keys()) - set(['fieldOfStudy','schoolName','deegreName'])
            for uw in unwanted: profile['education'][i].pop(uw)
    if profile['experience'] is not None:
        for i in range(len(profile['experience'])):
            unwanted = set(profile['experience'][i].keys()) - set(['companyName','title'])
            for uw in unwanted: profile['experience'][i].pop(uw)
    if profile['publications'] is not None:
        for i in range(len(profile['publications'])):
            unwanted = set(profile['publications'][i].keys()) - set(['date', 'name', 'publisher','description'])
            for uw in unwanted: profile['publications'][i].pop(uw)
    if profile['certifications'] is not None:
        for i in range(len(profile['certifications'])):
            unwanted = set(profile['certifications'][i].keys()) - set(['authority', 'name'])
            for uw in unwanted: profile['certifications'][i].pop(uw)
    try:
        if profile['birthDate'] is not None:
             profile['birthDate'] = (json.dumps(profile['birthDate']))
    except:
        pass
    if profile['geoLocation'] is not None:
         profile['geoLocation'] = (json.dumps(profile['geoLocation']))
    if profile['location'] is not None:
         profile['location'] = (json.dumps(profile['location']))

    profile['date'] = datetime.now().isoformat(timespec='seconds') + "Z"
    
    for k in profile:
        if type(profile[k])== type([]) and len(profile[k]) == 1 :
            profile[k].append({})
            
    print("profile data has been successfully processed")
    return [profile]

In [30]:
def querry_region():
    solr = pysolr.Solr("http://192.168.4.217:8983/solr/region", always_commit=True,  timeout=10)
    return solr.search('*').docs

In [31]:
def crawl_profile_ids():
    global region, current_region, search_count_per_acc
    offset= 0
    
    print('enetered crawl_profile function')
    regions = querry_region()
    print('list of regions have been retrieved')
    while search_count_per_acc[myAcc] <= Maximum_search:
        
        ## check for the offset
        if current_region in regions:
                offset = region[current_region]
        else:
                offset= 0
    
        profiles_id = api.search_people(regions= [current_region], offset= offset, limit= 20)
        print('{} profile ids are retirieved through linkedin api'.format(len(profiles_id)) )
        if len(profiles_id) == limit:
            offset += limit
        else:
            region[current_region] = offset
            current_region = regions[(randint(0,len(regions)))]['geoLocation']
            print('search region has changed')
            
        search_count_per_acc[myAcc] = len(profiles_id)
        index_profiles_id(profiles_id)
        print('program slept')
        sleep(randint(5,20))
        print('program has been awaken up')
        
    print('user account must be changed')
    change_acc()

In [32]:
def querry_profileId(count = 10):
    global start_profileId
    profileIds = list()
    solr = pysolr.Solr('http://192.168.4.217:8983/solr/profileId', always_commit=True, timeout=10)
    data = solr.search('*:*', rows=count, start= start_profileId).docs
    if len(data) == count:
        start_profileId += count
    else:
        print('more profileIds are needed')
        start_profileId += len(data)
        crawl_profile_ids()
        

    for i in data:
        profileIds.append(i['public_id'])
        
    return profileIds

In [38]:
def crawl_profile():
    global profile_count_per_acc
    
    print('enetered crawl_profile function')
    while profile_count_per_acc[myAcc] <= Maximum_profile:
        ids= querry_profileId(5)
        print('profile ids are retrieved')
        for id in ids:
            try:
                profile = api.get_profile(id)
                print('one profile data has fetched through linkedin API')
                index_region(profile)
                processed_profile = data_processing(profile)
                sleep(2)
                print(processed_profile)
                index_profile(processed_profile)
                profile_count_per_acc[myAcc] += 1
                print('program slept')
                sleep(randint(5,20))
                print('program has waken up')
            except:
                print('profile not requested')
                pass
            
    print('user account must be changed')
    sys.exit()
    change_acc()

In [39]:
querry_acc()
login()

INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/accounts/update/?commit=true' (post) with body '[{"user": ' in 0.748 seconds, with status 200
INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/accounts/select/?q=%2A&wt=json' (get) with body '' in 0.090 seconds, with status 200


accounts info are retrieved


INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/profileId/select/?q=%2A%3A%2A&rows=5&start=0&wt=json' (get) with body '' in 0.101 seconds, with status 200


user A is logged in successfully
enetered crawl_profile function
profile ids are retrieved
one profile data has fetched through linkedin API


INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/region/update/?commit=true' (post) with body '[{"geoLoca' in 0.578 seconds, with status 200


...

INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/region/update/?commit=true' (post) with body '[{"geoLoca' in 0.545 seconds, with status 200


geoLocation has successfully indexed
profile data has been successfully processed
[{'industryName': 'Information Technology & Services', 'lastName': 'sadeghi', 'locationName': 'Iran', 'student': False, 'geoCountryName': 'Iran', 'geoCountryUrn': 'urn:li:fs_geo:101934083', 'geoLocationBackfilled': False, 'elt': False, 'industryUrn': 'urn:li:fs_industry:96', 'firstName': 'neda', 'entityUrn': 'urn:li:fs_profile:ACoAABsbWlgBhmHXXtV4U-NkpLNSqKpjI5rzmk0', 'geoLocation': '{"geoUrn": "urn:li:fs_geo:106447486"}', 'geoLocationName': 'Tehran, Tehran', 'location': '{"basicLocation": {"countryCode": "ir"}}', 'headline': 'Looking for new job', 'displayPictureUrl': 'https://media-exp1.licdn.com/dms/image/C4D35AQEM_StCv-sbSw/profile-framedphoto-shrink_', 'img_200_200': '200_200/0/1598860036593?e=1642888800&v=beta&t=G8guHtjL4SC4dzRaQOXosASWCRsxnirga45t5CXgoK8', 'img_400_400': '400_400/0/1598860036593?e=1642888800&v=beta&t=8_UHXDk42_pccIn4_53OlflgM2WvN3byXSGul2HziNE', 'img_800_800': '800_800/0/1598860036

INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/person/update/?commit=true' (post) with body '[{"industr' in 1.034 seconds, with status 200


profile info has successfully indexed
program slept
profile not requested
user account must be changed


SystemExit: 

In [69]:
# querry_acc
# api = Linkedin(accounts[0]['user'], accounts[0]['password'])
profile = api.get_profile('zahra-st-954901153')
# print('one profile data has fetched through linkedin API')
index_region(profile)
processed_profile = data_processing(profile)
sleep(2)
# print(processed_profile)
index_profile(processed_profile)
processed_profile

INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/region/update/?commit=true' (post) with body '[{"geoLoca' in 0.532 seconds, with status 200


geoLocation has successfully indexed
profile data has been successfully processed


INFO:pysolr:Finished 'http://192.168.4.217:8983/solr/person/update/?commit=true' (post) with body '[{"industr' in 0.708 seconds, with status 200


profile info has successfully indexed


[{'industryName': 'Computer Software',
  'lastName': 'st',
  'locationName': 'Iran',
  'student': False,
  'geoCountryName': 'Iran',
  'geoCountryUrn': 'urn:li:fs_geo:101934083',
  'geoLocationBackfilled': False,
  'elt': False,
  'industryUrn': 'urn:li:fs_industry:4',
  'firstName': 'Zahra',
  'entityUrn': 'urn:li:fs_profile:ACoAACT5qBAB71fINA3Wf9at69z4Qja9ugosCmU',
  'geoLocation': '{"geoUrn": "urn:li:fs_geo:109197518"}',
  'geoLocationName': 'Sari County, Mazandaran',
  'location': '{"basicLocation": {"countryCode": "ir"}}',
  'headline': 'BackEnd Developer',
  'displayPictureUrl': 'https://media-exp1.licdn.com/dms/image/C4E03AQEBX81LrxLr1Q/profile-displayphoto-shrink_',
  'img_100_100': '100_100/0/1641115417403?e=1648080000&v=beta&t=e6FJIDAXwHAGeWs85d0UH41LXHn3Nt9VBw4kc8LT3SE',
  'img_200_200': '200_200/0/1641115417403?e=1648080000&v=beta&t=pmhEyzHdEwg_CfZX0SnHsR3-O4efe_R8EEIN4KbANgE',
  'img_400_400': '400_400/0/1641115417403?e=1648080000&v=beta&t=tYgKxaPlQ5YqBBFOMa1oDJj9NTwqaaU3W

In [66]:
## to check if there is any repeated profile_id
with open("tehran-pro.json", "r") as f:
    dt = json.loads(f.read())
    
for i in goo:
    flag = 0
    for j in dt:
        if i['public_id'] == j['public_id']:
            flag = 1
            break
    if flag == 0:
        dt.append(i)
        
json_data = json.dumps(dt)

## update the json file with new extracted data
with open("tehran-pro.json",'w') as f:
    f.write(json_data)

In [78]:
## saving the rigion urn id
def save_region(goo):
    if goo['geoLocation'] and goo['geoLocationName'] is not None:
        with open('location.json', 'r+') as rf:
            loc = json.loads(rf.read())
        if loc is not None:
            if goo['geoLocationName'] in loc[0] and goo['geoLocation'] not in loc[0].values():
                loc[0][goo['geoLocationName']].append(goo['geoLocation']['geoUrn'].split(':')[3]) 
            elif  goo['geoLocationName'] not in loc[0]:
                loc[0][goo['geoLocationName']]= goo['geoLocation']['geoUrn'].split(':')[3]
            else:
                pass

        with open('location.json', 'w') as wf:
            json_data = json.dump(loc, wf)
    else:
        pass
       

In [14]:
# with open("tehran-pro.json", "r") as f:
#     dt = json.loads(f.read())
# with open('location.json','w') as file:
#     json.dump([{}],file)

with open('location.json', 'w') as wf:
    json_data = json.dump(loc, wf)

index_profiles_id(dt)

In [3]:
import time
print('hello')
time.sleep(10)
print('hello-again')

hello
hello-again


In [114]:
index_region(profile):    
    solr = pysolr.Solr("http://192.168.4.217:8983/solr/region", always_commit=True, timeout=100)
    
    ## to check if the city and geoUrn have indexed or not, then decide to whether update field or add
    result = pd.DataFrame(solr.search('*').docs)
    if profile['geoLocationName'] in list(result['geoLocationName']) and profile['geoLocation']['geoUrn'].split(':')[3] not in list(result['geoLocation']):
        print(list(result['geoLocation']))
        solr.add([{"geoLocationName": profile['geoLocationName'],
                      "geoLocation": {"add": [profile['geoLocation']['geoUrn'].split(':')[3]] }
                }])
    elif profile['geoLocationName'] not in list(result['geoLocationName']):
        solr.add([{"geoLocationName": profile['geoLocationName'],
                      "geoLocation":  profile['geoLocation']['geoUrn'].split(':')[3] 
                }])
    elif profile['geoLocationName'] in list(result['geoLocationName']) and profile['geoLocation']['geoUrn'].split(':')[3] in list(result['geoLocation']):
        pass

In [47]:
<field name="id" type="string" indexed="true" stored="true" required="true" multiValued="false" />



[{'urn_id': 'ACoAABdBEMQBZAF_nGzHDZpIfB498RuqedxjGHk',
  'distance': 'DISTANCE_2',
  'public_id': 'fahime-firoozpour',
  'tracking_id': '390140100'},
 {'urn_id': 'ACoAADOrA4ABNX2ouTmLFR5wHiOlESLxjrYITrs',
  'distance': 'DISTANCE_2',
  'public_id': 'sabrina-mokhtari',
  'tracking_id': '866845568'},
 {'urn_id': 'ACoAACOmMJYBbFwZps8cx_Vb98R8C3IqFAEtUYA',
  'distance': 'DISTANCE_2',
  'public_id': 'hannah-g-246373148',
  'tracking_id': '598093974'},
 {'urn_id': 'ACoAACt21okB46JWVGMZ5KKSNcM2eJyQ6YY9hF8',
  'distance': 'DISTANCE_2',
  'public_id': 'fateme-alizadeh-775263184',
  'tracking_id': '729208457'},
 {'urn_id': 'ACoAABOHFRgBLMYoGD3wrOKKH-hI-WiAAdlC3cE',
  'distance': 'DISTANCE_2',
  'public_id': 'sanaztaheri',
  'tracking_id': '327619864'},
 {'urn_id': 'ACoAACrXLloBgGNRfZIza23TptJaG5A4FF6JLCM',
  'distance': 'DISTANCE_2',
  'public_id': 'mahsagoudarzi',
  'tracking_id': '718745178'},
 {'urn_id': 'ACoAABGGgtgBX1u7oDx1mmL2OL-5nAUf9fIDNQs',
  'distance': 'DISTANCE_2',
  'public_id': 'far

In [ ]:
"lastName": profile['lastName'],
              "entityUrn": profile['entityUrn'],
              "firstName":profile['firstName'],